In [1]:
from bs4 import BeautifulSoup
import urllib.request

from selenium import webdriver
import pandas as pd
import numpy as np

from collections import OrderedDict

import time
from tqdm import tqdm

from nltk.tokenize import word_tokenize



In [2]:
vocab = {} 
preprocessed_sentences = []

# **공시가 발생한 회사명, 종목코드 추출**

1. step one : 첨부서류 코드 모으기
2. step two : 첨부서류에 들어가서 회사명, 종목코드 추출

## **step one - 첨부서류 코드 수집**

In [3]:
docscode_2 = []
cpy_info = []
link_box = []
date = []
checkbox = [10,29] # 11,10번은 따로 23소송도 따로

""" 1:가족친화/녹색경영정보 3:기술도입 4:기술성장기업 14:벌금/과징금/행정조치 15:부도/은행거래정지 19:사채/대출원리금미지급 24:손상차손등 27:어음 위·변조 63:해산/파산 65:회계처리기준위반등 67:횡령
11:대표이사변경 29:영업정지/조업중단등 10:담보제공/채무보증 23:소송 37:재해발생 40:제품수거/파기 50:증자/감자 54:차입 """

"""추출 리스트 1,3,4,14,15,19,24,27,63,65,67 """
start = 2000
end = 2020
section = 3



In [4]:
# 회사명 추출
url = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
driver.get(url)
driver.find_element_by_id('rKosdaq').click() # 코스닥
driver.find_element_by_xpath('//*[@id="business"]/option[4]').click() # 산업군
driver.find_element_by_xpath('//*[@id="bfrDsclsType"]').click() # 공시유형 초기화 체크 해제
driver.find_element_by_xpath('//*[@id="dsclsType01"]').click() # 수시공시 클릭

for i in range(len(checkbox)):
    driver.find_element_by_xpath('//*[@id="dsclsLayer01_{num}"]'.format(num = checkbox[i])).click() # checklist
    #time.sleep(0.1)

time.sleep(0.1)    
driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # show 100 case
time.sleep(0.1)

for i in tqdm(range((end - start)//section + 1)):
    
    sty = driver.find_element_by_id("fromDate")
    #time.sleep(0.1)
    sty.clear()
    #time.sleep(0.1)
    sty.send_keys("{fromD}-01-01".format(fromD= start))
    #time.sleep(0.1)

    edy = driver.find_element_by_id("toDate")
    #time.sleep(0.1)
    edy.clear()
    #time.sleep(0.1)
    edy.send_keys("{toD}-12-31".format(toD = start + 2))

    driver.find_element_by_xpath('//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
    time.sleep(2)

    page = driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/em').text # 총 몇 페이지
    page = page.replace(',','')
    page = int(page)
    page_ = page 
    print('{start}-{end_}년도의 해당 공시자료 수는 {page}입니다.'.format(start=start,end_=start+section-1, page=page))
    
    if page <= 1000:
        # 남은 페이지가 100이 넘을 때
        while page_ >=100:
            print('남은 페이지 수는 {num}입니다.'.format(num = page_))
            for i in range(100): # 공시 날짜 수집
                date_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[2]'.format(num = i + 1)).text
                date.append(date_value)
                #time.sleep(0.3)
            print('잔여 날짜 RESPONSE 200')

            for i in range(100): # 문서 코드
                onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]/a'.format(num = i + 1)).get_attribute('onclick')
                onclick_value = onclick_value[18:32]
                #time.sleep(0.3)
                docscode_2.append(onclick_value)
            print('잔여 문서 코드 RESPONSE 200')
            print('공시 날짜 수:{date} 문서 코드 수:{docscode_2}'.format(date=len(date),docscode_2=len(docscode_2)))
            time.sleep(0.2)
            driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[1]/a[{num}]'.format(num = page//100 + 4)).click() # 페이지 넘기는 코드
            time.sleep(2)
            page_ = page_ - 100
    else:
        print('공시문서가 1000개가 넘어 오류가 발생합니다!!')

    # 남은 페이지가 100보다 작을 떄     
    """ if page_ < 100: """
    print('남은 페이지 수는 {num}입니다. 수고했어'.format(num = page_))

    for i in range(page_): # 공시 날짜 수집
        date_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[2]'.format(num = i + 1)).text
        date.append(date_value)
        #time.sleep(0.3)
    print('잔여 날짜 RESPONSE 200')

    for i in range(page_): # 문서 코드
        onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]/a'.format(num = i + 1)).get_attribute('onclick')
        onclick_value = onclick_value[18:32]
        #time.sleep(0.3)
        docscode_2.append(onclick_value)
    print('잔여 문서 코드 RESPONSE 200')

    print('공시 날짜 수:{date} 문서 코드 수:{docscode_2}'.format(date=len(date),docscode_2=len(docscode_2)))

    start += 3


  0%|          | 0/7 [00:00<?, ?it/s]

2000-2002년도의 해당 공시자료 수는 184입니다.
남은 페이지 수는 184입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:100 문서 코드 수:100
남은 페이지 수는 84입니다. 수고했어
잔여 날짜 RESPONSE 200


 14%|█▍        | 1/7 [00:26<02:39, 26.51s/it]

잔여 문서 코드 RESPONSE 200
공시 날짜 수:184 문서 코드 수:184
2003-2005년도의 해당 공시자료 수는 386입니다.
남은 페이지 수는 386입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:284 문서 코드 수:284
남은 페이지 수는 286입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:384 문서 코드 수:384
남은 페이지 수는 186입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:484 문서 코드 수:484
남은 페이지 수는 86입니다. 수고했어
잔여 날짜 RESPONSE 200


 29%|██▊       | 2/7 [01:10<03:03, 36.76s/it]

...

잔여 문서 코드 RESPONSE 200
공시 날짜 수:2363 문서 코드 수:2363
2015-2017년도의 해당 공시자료 수는 605입니다.
남은 페이지 수는 605입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:2463 문서 코드 수:2463
남은 페이지 수는 505입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:2563 문서 코드 수:2563
남은 페이지 수는 405입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:2663 문서 코드 수:2663
남은 페이지 수는 305입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:2763 문서 코드 수:2763
남은 페이지 수는 205입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:2863 문서 코드 수:2863
남은 페이지 수는 105입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:2963 문서 코드 수:2963
남은 페이지 수는 5입니다. 수고했어
잔여 날짜 RESPONSE 200


 86%|████████▌ | 6/7 [05:54<01:06, 66.52s/it]

잔여 문서 코드 RESPONSE 200
공시 날짜 수:2968 문서 코드 수:2968
2018-2020년도의 해당 공시자료 수는 750입니다.
남은 페이지 수는 750입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3068 문서 코드 수:3068
남은 페이지 수는 650입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3168 문서 코드 수:3168
남은 페이지 수는 550입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3268 문서 코드 수:3268
남은 페이지 수는 450입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3368 문서 코드 수:3368
남은 페이지 수는 350입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3468 문서 코드 수:3468
남은 페이지 수는 250입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3568 문서 코드 수:3568
남은 페이지 수는 150입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:3668 문서 코드 수:3668
남은 페이지 수는 50입니다. 수고했어
잔여 날짜 RESPONSE 200


100%|██████████| 7/7 [07:28<00:00, 64.13s/it]

잔여 문서 코드 RESPONSE 200
공시 날짜 수:3718 문서 코드 수:3718


## **step two - 해당 공시문서가 발생한 회사명, 종목코드 추출**

In [5]:
dftcodebox = []

for j in range(len(docscode_2)):
        try:
                url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[j])
                driver.get(url)
                time.sleep(0.3)
                cpy_nameAndcode = driver.find_element_by_xpath('//*[@id="frm"]/header/h1').text
                cpy_info.append(cpy_nameAndcode)
        except ValueError as er:
                print(er,'문서코드와 url주소 매칭에 실패하였습니다. 임시 코드 리스트에 해당 코드를 보관합니다.')
                dftcodebox.append(docscode_2[j])
driver.close()

# **공시문서 텍스트 수집**

In [6]:
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
for i in range(int(len(docscode_2)/3)): # 텍스트 링크 추출
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)

time.sleep(1)

for i in range(int(len(docscode_2)/3), int(len(docscode_2)/3*2)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)    

time.sleep(1)

for i in range(int(len(docscode_2)/3*2), len(docscode_2)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)   

driver.close()
len(link_box)


3718

## **공시 문서 URL 리스트에 결측지 유무 확인**

In [7]:
cnt = 0
dftbox = []
for i in range(len(link_box)):
    
    if link_box[i] == '':
        print('index:{num} 주소값에는 결측지가 전달되었습니다.'.format(num = i))
        dftbox.append(i)
        cnt += 1
print('총 결측지는 {num}개 입니다.'.format(num = cnt))

index:694 주소값에는 결측지가 전달되었습니다.
index:1657 주소값에는 결측지가 전달되었습니다.
index:2149 주소값에는 결측지가 전달되었습니다.
index:2162 주소값에는 결측지가 전달되었습니다.
index:2187 주소값에는 결측지가 전달되었습니다.
index:2283 주소값에는 결측지가 전달되었습니다.
index:2370 주소값에는 결측지가 전달되었습니다.
index:2509 주소값에는 결측지가 전달되었습니다.
index:2857 주소값에는 결측지가 전달되었습니다.
index:2865 주소값에는 결측지가 전달되었습니다.
index:3213 주소값에는 결측지가 전달되었습니다.
index:3385 주소값에는 결측지가 전달되었습니다.
총 결측지는 12개 입니다.


In [8]:
driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')

for i in dftbox:
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={num}&docno=&viewerhost=&viewerport='.format(num = docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link_box[i] = driver.find_element_by_id('docLocPath').get_attribute('value')

driver.close()

## **결측치 재확인**

In [9]:
cnt_ = 0
for i in range(len(link_box)):
    
    if link_box[i] == '':
        print('index:{num} 주소값에는 결측지가 전달되었습니다.'.format(num = i))
        cnt_ += 1
print('총 결측지는 {num}개 입니다.'.format(num = cnt_))

총 결측지는 0개 입니다.


## **공시 text 추출**

In [10]:
textbox = []

for i in tqdm(range(len(link_box))):
    html = urllib.request.urlopen(link_box[i])

    soup = BeautifulSoup(html, 'html.parser').text
    textbox.append(soup)

print(len(link_box))    


100%|██████████| 3718/3718 [08:53<00:00,  6.97it/s]

3718


# **회사명, 코드 및 텍스트 DF, csv로 만들기**

In [11]:
cpyname = []
cpycode = []

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

con1 = OrderedDict([("(", ""), (")", "")])

In [12]:
for i in range(len(cpy_info)):
    infm = cpy_info[i].split(' ')
    cpyname.append(infm[0])

    cpycode.append(replace_all(infm[1] , con1))

In [13]:
text_df = pd.DataFrame({'회사명': cpyname,
                        '거래소코드' : cpycode,
                        '공시발생일' : date,
                        'text' : textbox, 
                        'link' : link_box})
text_df

,회사명,거래소코드,공시발생일,text,link
0,데코,013650,2002-12-31 15:11,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/12/31/K00...
1,다휘,055250,2002-12-30 14:18,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/12/30/K00...
2,이수페타시스,007660,2002-12-23 13:56,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/12/23/K00...
3,인지디스플레,037330,2002-12-14 14:00,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/12/14/K00...
4,인지디스플레,037330,2002-12-14 13:28,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/12/14/K00...
...,...,...,...,...,...
3713,우리바이오,082850,2018-01-03 15:54,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2018/01/03/000...
3714,바이오빌,065940,2018-01-02 17:56,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2018/01/02/000...
3715,싸이맥스,160980,2018-01-02 16:22,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2018/01/02/000...
3716,삼표시멘트,038500,2018-01-02 15:01,\n\n\n\n\n\n:: 70656_대표이사 변경\n\n\n\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2018/01/02/000...


## **텍스트 정보 CSV**

In [14]:
checkbox

[11]

In [15]:
text_df.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Code\JH\data\0020공시11.csv', encoding='utf-8-sig')

# **NLP**

## **토큰화**

In [92]:
wordbox = []

for i in tqdm(range(len(textbox))):
    wordbox.append(word_tokenize(textbox[i]))

wordbox

100%|██████████| 119/119 [00:00<00:00, 526.69it/s]


In [95]:
len(wordbox)

119

In [97]:
for i in tqdm(range(len(wordbox))):
    # 단어 토큰화
    result = []
    for word in wordbox[i]:        
        if word not in vocab:
            vocab[word] = 0 
        vocab[word] += 1
    preprocessed_sentences.append(result) 

100%|██████████| 119/119 [00:00<00:00, 23941.01it/s]


In [107]:
sorted(vocab.items(), key= lambda x :x[1], reverse= True)

[('.', 4778),
 (')', 3299),
 ('(', 2965),
 ('-', 1420),
 (':', 1342),
 ('원', 1207),
 ('1', 785),
 ('2', 775),
 ('상기', 719),
 (',', 679),
 ('3', 622),
 ('및', 508),
 ('횡령', 502),
 ('4', 498),
 ('진행사항', 438),
 ('등', 424),
 ('자기자본', 400),
 ('혐의', 372),
 ('기타', 358),
 ('사항', 353),
 ('%', 351),
 ('참고할', 343),
 ('투자판단에', 341),
 ('자기자본대비', 330),
 ('5', 322),
 ('발생', 309),
 ('예정입니다', 281),
 ('대표이사', 278),
 ('횡령ㆍ배임', 263),
 ('금액', 261),
 ('미해당', 240),
 ('혐의발생', 237),
 ('대출원리금', 237),
 ('발생금액', 234),
 ('확인일자', 233),
 ('연체', 233),
 ('대기업해당여부', 232),
 ('자기자본은', 229),
 ('※관련공시', 228),
 ('6', 211),
 ('대한', 209),
 ('주', 202),
 ('사고발생내용', 200),
 ('따라', 200),
 ('이자', 194),
 ('*', 181),
 ('향후대책', 173),
 ('배임', 171),
 ('전', 169),
 ('합계', 169),
 ('발생일자', 163),
 ('연체사실', 158),
 ('확인일자는', 157),
 ('당사는', 154),
 ('사고발생일자', 145),
 ('중', 143),
 ('관련', 142),
 ('대책', 138),
 ('前', 135),
 ('가', 133),
 ('벌금', 132),
 ('금액입니다', 126),
 ('본', 125),
 ('2019년', 123),
 ('관련하여', 122),
 ('향후', 122),
 ('후', 121),
 ('대출금', 120)